In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler   # 归一化
from sklearn.metrics import r2_score,mean_absolute_error

import tensorflow as tf
from tensorflow.keras import Sequential, layers, utils

import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'numpy'

### 第1步：加载数据集、预处理

In [ ]:
# 读取数据集

dataset = pd.read_csv('data/附件1-区域15分钟负荷数据.csv')

: 

In [ ]:
# 显示shape 
  
dataset.shape

: 

In [1]:
# 默认显示前5行

dataset.head()

NameError: name 'dataset' is not defined

In [ ]:
# 显示数据描述

dataset.describe()

: 

In [ ]:
# 显示字段数据类型

dataset.dtypes

: 

In [ ]:
# 将字段Datetime数据类型转换为日期类型

dataset['Datetime'] = pd.to_datetime(dataset['Datetime'], format="%Y-%m-%d %H:%M:%S")

: 

In [ ]:
# 再次查看字段的数据类型

dataset.dtypes

: 

In [ ]:
# 将字段Datetime设置为索引列
# 目的：后续基于索引来进行数据集的切分

dataset.index = dataset.Datetime

: 

In [ ]:
# 显示默认前5行

dataset.head()

: 

In [ ]:
# 将原始的Datetime字段列删除

dataset.drop(columns=['Datetime'], axis=1, inplace=True)

: 

In [ ]:
# 默认显示前5行

dataset.head()

: 

In [ ]:
# 可视化显示Power的数据分布情况

dataset['Power'].plot(figsize=(22,10))
plt.show()

: 

In [ ]:
# 数据进行归一化

scaler = MinMaxScaler()

dataset['Power'] = scaler.fit_transform(dataset['Power'].values.reshape(-1, 1))

: 

In [ ]:
# 均值为0，标准差为1

dataset.head()

: 

In [ ]:
# 可视化显示Power的数据分布情况

dataset['Power'].plot(figsize=(16,8))
plt.show()

: 

### 第2步：特征工程

In [ ]:
# 功能函数：构造特征数据集和标签集

def create_new_dataset(dataset, seq_len = 12):
    '''基于原始数据集构造新的序列特征数据集
    Params:
        dataset : 原始数据集
        seq_len : 序列长度（时间跨度）
    
    Returns:
        X, y
    '''
    X = [] # 初始特征数据集为空列表
    y = [] # 初始标签数据集为空列表
    
    start = 0 # 初始位置
    end = dataset.shape[0] - seq_len # 截止位置
    
    for i in range(start, end): # for循环构造特征数据集
        sample = dataset[i : i+seq_len] # 基于时间跨度seq_len创建样本
        label = dataset[i+seq_len] # 创建sample对应的标签
        X.append(sample) # 保存sample
        y.append(label) # 保存label
    
    # 返回特征数据集和标签集
    return np.array(X), np.array(y)

: 

In [ ]:
# 功能函数：基于新的特征的数据集和标签集，切分：X_train, X_test

def split_dataset(X, y, train_ratio=0.8):
    '''基于X和y，切分为train和test
    Params:
        X : 特征数据集
        y : 标签数据集
        train_ratio : 训练集占X的比例
    
    Returns:
        X_train, X_test, y_train, y_test
    '''
    X_len = len(X) # 特征数据集X的样本数量
    train_data_len = int(X_len * train_ratio) # 训练集的样本数量
    
    X_train = X[:train_data_len] # 训练集
    y_train = y[:train_data_len] # 训练标签集
    
    X_test = X[train_data_len:] # 测试集
    y_test = y[train_data_len:] # 测试集标签集
    
    # 返回值
    return X_train, X_test, y_train, y_test

: 

In [ ]:
# 功能函数：基于新的X_train, X_test, y_train, y_test创建批数据(batch dataset)

def create_batch_data(X, y, batch_size=32, data_type=1):
    '''基于训练集和测试集，创建批数据
    Params:
        X : 特征数据集
        y : 标签数据集
        batch_size : batch的大小，即一个数据块里面有几个样本
        data_type : 数据集类型（测试集表示1，训练集表示2）
   
    Returns:
        train_batch_data 或 test_batch_data
    '''
    if data_type == 1: # 测试集
        dataset = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y))) # 封装X和y，成为tensor类型
        test_batch_data = dataset.batch(batch_size) # 构造批数据
        # 返回
        return test_batch_data
    else: # 训练集
        dataset = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y))) # 封装X和y，成为tensor类型
        train_batch_data = dataset.cache().shuffle(1000).batch(batch_size) # 构造批数据
        # 返回
        return train_batch_data

: 

In [ ]:
# ① 原始数据集

dataset_original = dataset

: 

In [ ]:
print("原始数据集: ", dataset_original.shape)

: 

In [ ]:
# ② 构造特征数据集和标签集，seq_len序列长度为12小时

SEQ_LEN = 36 # 序列长度

X, y = create_new_dataset(dataset_original.values, seq_len = SEQ_LEN)

: 

In [ ]:
X.shape

: 

In [ ]:
y.shape

: 

In [ ]:
# 样本1 - 特征数据

X[0]

: 

In [ ]:
# 样本1 - 标签

y[0]

: 

In [ ]:
# ③ 数据集切分

X_train, X_test, y_train, y_test = split_dataset(X, y, train_ratio=0.8)

: 

In [ ]:
X_train.shape

: 

In [ ]:
X_test.shape

: 

In [ ]:
y_train.shape

: 

In [ ]:
y_test.shape

: 

In [ ]:
# ④ 基于新的X_train, X_test, y_train, y_test创建批数据(batch dataset)

# 测试批数据

test_batch_dataset = create_batch_data(X_test, y_test, batch_size=256, data_type=1)

: 

In [ ]:
# 训练批数据

train_batch_dataset = create_batch_data(X_train, y_train, batch_size=256, data_type=2)

: 

### 第3步：构建模型

In [ ]:
model = Sequential([
    layers.LSTM(16, input_shape=(SEQ_LEN, 1)),
    layers.Dense(1)
])

: 

In [ ]:
# 显示模型结构

utils.plot_model(model)

: 

In [ ]:
# 定义 checkpoint，保存权重文件

file_path = "best_checkpoint.hdf5"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=file_path, 
                                                         monitor='loss', 
                                                         mode='min', 
                                                         save_best_only=True,
                                                         save_weights_only=True)

: 

### 第4步：模型编译、训练、验证

In [ ]:
# 模型编译

model.compile(optimizer='adam', loss="mae")

: 

In [ ]:
# 模型训练

history = model.fit(train_batch_dataset,
          epochs=100,
          validation_data=test_batch_dataset,
          callbacks=[checkpoint_callback])

: 

In [ ]:
# 显示 train loss 和 val loss

plt.figure(figsize=(16,8))
plt.plot(history.history['loss'],'#0000ff',label='train loss')
plt.plot(history.history['val_loss'],'r',label='val loss')
plt.title("LOSS")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='best')
plt.show()

: 

In [ ]:
# 模型验证

test_pred = model.predict(X_test, verbose=1)

: 

In [ ]:
test_pred.shape # 预测结果的shape

: 

In [ ]:
y_test.shape # 真值标签的shape

: 

In [ ]:
# 计算r2

score = r2_score(y_test, test_pred)

: 

In [ ]:
print("r^2 的值： ", score)

: 

In [ ]:
# 绘制模型验证结果

plt.figure(figsize=(16,8))
plt.plot(y_test, label="True label")
plt.plot(test_pred, label="Pred label")
plt.title("True vs Pred")
plt.legend(loc='best')
plt.show()

: 

In [ ]:
# 绘制test中前100个点的真值与预测值

y_true = y_test[:100]
y_pred = test_pred[:100]

fig, axes = plt.subplots(2, 1, figsize=(16,8))
axes[0].plot(y_true, marker='o', color='red')
axes[1].plot(y_pred, marker='*', color='blue')
plt.show()

: 

### 第5步：模型测试

#### ① 预测1个样本

In [ ]:
# 选择test中的最后一个样本
sample = X_test[-1]

sample.shape

: 

In [ ]:
sample = sample.reshape(1, sample.shape[0], 1)

sample.shape

: 

In [ ]:
# 模型预测

sample_pred = model.predict(sample)

sample_pred

: 

#### ② 预测后续20个点的值 

In [ ]:
ture_data = X_test[-1] # 真实test的最后20个数据点

: 

In [ ]:
ture_data

: 

In [ ]:
ture_data.shape

: 

In [ ]:
list(ture_data[:,0])

: 

In [ ]:
def predict_next(model, sample, epoch=20):
    temp1 = list(sample[:,0])
    for i in range(epoch):
        sample = sample.reshape(1, SEQ_LEN, 1)
        pred = model.predict(sample)
        value = pred.tolist()[0][0]
        temp1.append(value)
        sample = np.array(temp1[i+1 : i+SEQ_LEN+1])
    return temp1

: 

In [ ]:
preds = predict_next(model, ture_data, 20)

: 

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(preds, color='red', label='Prediction')
plt.plot(ture_data, color='blue', label='Truth')
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend(loc='best')
plt.show()

: 

: 